In [1]:
import json
import os
import pandas as pd
from pathlib import Path
from urllib.parse import unquote

In [2]:
def consolidar_dataset(caminho_raw):
    lista_final = []
    caminho_base = Path(caminho_raw)
    
    # Percorre todas as pastas de alunos
    for pasta_aluno in caminho_base.iterdir():
        if not pasta_aluno.is_dir():
            continue
            
        # Tenta encontrar a pasta de keypoints (tratando variações de nome)
        # Procura por 'Key_points' ou 'keypoints'
        caminho_kp = None
        for sub in ['Key_points', 'keypoints', 'Keypoints']:
            if (pasta_aluno / sub).exists():
                caminho_kp = pasta_aluno / sub
                break
        
        if caminho_kp:
            print(f"Processando: {pasta_aluno.name}")
            arquivos_encontrados = 0
            
            # Itera sobre TODOS os arquivos na pasta, independente da extensão
            for arquivo_json in caminho_kp.iterdir():
                if arquivo_json.is_dir() or arquivo_json.name.startswith('.'):
                    continue
                
                try:
                    with open(arquivo_json, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                    
                    # Se o JSON for uma lista, tratamos como várias tarefas
                    # Se for um dicionário, colocamos em uma lista para uniformizar
                    tasks = data if isinstance(data, list) else [data]
                    
                    for task in tasks:
                        # Pega o nome da imagem com segurança
                        img_path = task.get('task', {}).get('data', {}).get('img', '')
                        nome_imagem = unquote(img_path.split('/')[-1])
                        
                        results = task.get('result', [])
                        if not results:
                            continue
                            
                        temp_map = {}
                        for res in results:
                            res_id = res['id']
                            if res_id not in temp_map:
                                temp_map[res_id] = {'aluno': pasta_aluno.name, 'imagem': nome_imagem}
                            
                            val = res.get('value', {})
                            # Captura coordenadas
                            if res['type'] == 'keypointlabels':
                                temp_map[res_id]['ponto'] = val.get('keypointlabels', ['unknown'])[0]
                                temp_map[res_id]['x'] = val.get('x')
                                temp_map[res_id]['y'] = val.get('y')
                            
                            # Captura visibilidade
                            elif res['type'] == 'choices':
                                temp_map[res_id]['visibilidade'] = val.get('choices', [None])[0]

                        # Adiciona ao dataset global apenas os pontos válidos
                        for item in temp_map.values():
                            if 'ponto' in item and item['ponto'] != 'cow':
                                lista_final.append(item)
                    
                    arquivos_encontrados += 1
                except Exception as e:
                    print(f"Erro ao ler {arquivo_json}: {e}")
            
            print(f"   -> {arquivos_encontrados} arquivos lidos com sucesso.")
        else:
            print(f"Pulando: {pasta_aluno.name} (Pasta de keypoints não encontrada)")

    return pd.DataFrame(lista_final)

In [3]:
# Execução
df = consolidar_dataset('../data/raw')

if not df.empty:
    # Garante que a pasta processed existe
    Path('../data/processed').mkdir(parents=True, exist_ok=True)
    df.to_csv('../data/processed/dataset_completo.csv', index=False)
    print(f"\nSucesso! {len(df)} pontos consolidados.")
else:
    print("\nO DataFrame continua vazio. Verifique se os arquivos dentro das pastas são realmente JSONs válidos.")

df.head(10)

Processando: 13 - Juliano - Complemento (pronto)
   -> 5 arquivos lidos com sucesso.
Processando: 08 - Eduardo Braga (pronto)
   -> 44 arquivos lidos com sucesso.
Processando: 14 - Lucas (pronto)
   -> 44 arquivos lidos com sucesso.
Processando: 05 - Camilla (pronto)
   -> 44 arquivos lidos com sucesso.
Processando: 06 - Carlos Augusto (pronto)
   -> 44 arquivos lidos com sucesso.
Processando: 15 - Marcelo Azevedo (pronto)
   -> 88 arquivos lidos com sucesso.
Processando: 23 - Ronen (pronto)
   -> 44 arquivos lidos com sucesso.
Processando: 17 - Miguel (pronto)
   -> 49 arquivos lidos com sucesso.
Processando: 21 - Raquel (pronto)
   -> 47 arquivos lidos com sucesso.
Processando: 07 - Carlos Henrique (pronto)
   -> 44 arquivos lidos com sucesso.
Processando: 11 - Gustavo (pronto)
   -> 47 arquivos lidos com sucesso.
Processando: 03 - Raony (pronto)
   -> 44 arquivos lidos com sucesso.
Processando: 01 - Thales (pronto)
   -> 45 arquivos lidos com sucesso.
Processando: 20 - Rafael (pront

,aluno,imagem,ponto,x,y,visibilidade
0,13 - Juliano - Complemento (pronto),RLC4_00_20260114221438_baia10_RLC4.jpg,withers,10.714656,49.504950,Visível
1,13 - Juliano - Complemento (pronto),RLC4_00_20260114221438_baia10_RLC4.jpg,back,39.367333,49.834983,Visível
2,13 - Juliano - Complemento (pronto),RLC4_00_20260114221438_baia10_RLC4.jpg,hook up,67.658841,10.231023,Visível
3,13 - Juliano - Complemento (pronto),RLC4_00_20260114221438_baia10_RLC4.jpg,hook down,69.946240,85.148515,Visível
4,13 - Juliano - Complemento (pronto),RLC4_00_20260114221438_baia10_RLC4.jpg,hip,71.752081,48.184818,Visível
5,13 - Juliano - Complemento (pronto),RLC4_00_20260114221438_baia10_RLC4.jpg,tail head,85.837640,47.524752,Visível
6,13 - Juliano - Complemento (pronto),RLC4_00_20260114221438_baia10_RLC4.jpg,pin up,92.218278,27.722772,Visível
7,13 - Juliano - Complemento (pronto),RLC4_00_20260114221438_baia10_RLC4.jpg,pin down,91.616331,70.957096,Visível
8,13 - Juliano - Complemento (pronto),RLC4_00_20260114221037_baia1_RLC4.jpg,withers,8.784597,32.819787,Visível
9,13 - Juliano - Complemento (pronto),RLC4_00_20260114221037_baia1_RLC4.jpg,back,35.138387,34.164860,Visível
